## Start Up

In [17]:
import io
import os
import os.path
import re
import matplotlib.pyplot as plt
import cv2
import csv
import numpy as np
from PIL import Image
from io import BytesIO
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from googleapiclient.http import MediaIoBaseDownload
import torch
import torchvision

In [133]:
SCOPES = ['https://www.googleapis.com/auth/drive.file']
MIME_TYPE = 'application/vnd.google-apps.document'
APPLICATION_NAME = 'ipa-google-drive-api-client'

def get_service():

    # credentialの取得
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'google-drive-api.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
        
    # serviceの取得
    service = build('drive', 'v3', credentials=creds) 
    
    return service

def read_ocr(service, input_file, lang='jp'):
    # ファイルのアップロード

    # ローカルファイルの定義
    media_body = MediaFileUpload(input_file, mimetype=MIME_TYPE, resumable=True)

    # Google Drive上のファイル名
    newfile = 'output.pdf'

    body = {
        'name': newfile,
        'mimeType': MIME_TYPE
    }

    # 　creat関数でファイルアップロード実行
    # 同時にOCR読み取りも行う
    output = service.files().create(
        body=body,
        media_body=media_body,
        # ここで読み込み先言語の指定を行う
        ocrLanguage=lang,
    ).execute()

    # テキストファイルのダウンロード

    # リクエストオブジェクト生成
    request = service.files().export_media(
        fileId=output['id'],
        mimeType="text/plain"
    )

    
        # テキストの取得
    with open(output_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # 読み取り結果のリストを返す
    return lines[1:]


service = get_service()

In [100]:
def process_text_file(text_file):
    output_dir = "C:/Users/covid/text_recognition/output"
    if os.path.exists(output_dir):
        file_list = [f for f in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, f))]
        for file_name in file_list:
            file_path = os.path.join(output_dir, file_name)
            os.remove(file_path)
    else:
        os.makedirs(output_dir)

    with open(text_file, 'r') as file:
        lines = file.readlines()

        lines = sorted(lines, key=lambda line: float(line.split()[1]))

        for i, line in enumerate(lines):
            line = line.strip()
            values = line.split()

            if len(values) == 5:
                object_class = values[0]
                a = float(values[1])
                b = float(values[2])
                c = float(values[3])
                d = float(values[4])

                # Calculate coordinates and dimensions
                x_center = int(wid * a)
                y_center = int(hei * b)
                width = int(wid * c)
                height = int(hei * d)

                x_min = x_center - width // 2
                y_min = y_center - height // 2
                x_max = x_center + width // 2
                y_max = y_center + height // 2

                output_filename = os.path.join(output_dir, f'book{i+1}.jpg')
                index = 1
                while os.path.exists(output_filename):
                    output_filename = os.path.join(output_dir, f'book{i+1}_{index}.jpg')
                    index += 1

                # Crop and save the image
                cropped = image.crop((x_min, y_min, x_max, y_max))
                cropped.save(output_filename)


## Execution

In [108]:
# 画像ファイルの読み込み
image = cv2.imread('book11.jpg')

# 画像の高さと幅を取得
height, width, _ = image.shape

# サイズを表示
print("画像の幅:", width)
print("画像の高さ:", height)

画像の幅: 474
画像の高さ: 148


In [124]:
cap = cv2.VideoCapture(0)

ret, frame = cap.read()

if ret:#カメラと本棚の距離→0.6m
    x = 0  # トリミングの左上のX座標
    y = 90  # トリミングの左上のY座標
    width = 640  # トリミングする幅
    height = 200  # トリミングする高さ
    
    cropped_frame = frame[y:y+height, x:x+width]

    cv2.imwrite("C:/Users/covid/text_recognition/yolov7/input.png", cropped_frame)

    cap.release()

In [125]:
cd "C:\Users\covid\text_recognition\yolov7"

C:\Users\covid\text_recognition\yolov7


In [126]:
# ディレクトリのパス
directory_path = "C:/Users/covid/text_recognition/yolov7/runs/detect/"

# ディレクトリ内のサブディレクトリのリストを取得
subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

# サブディレクトリの中で一番新しいものを取得
newest_subdirectory = max(subdirectories, key=lambda d: os.path.getctime(os.path.join(directory_path, d)))

# 最新のサブディレクトリのパスを作成
newest_subdirectory_path = os.path.join(directory_path, newest_subdirectory)

In [127]:
!python detect.py --source C:/Users/covid/text_recognition/yolov7/input.png --weights yolov7-e6e.pt --conf 0.25 --img-size 1280 --device 0 --save-txt

Namespace(weights=['yolov7-e6e.pt'], source='C:/Users/covid/text_recognition/yolov7/input.png', img_size=1280, conf_thres=0.25, iou_thres=0.45, device='0', view_img=False, save_txt=True, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

25 books, Done. (16.0ms) Inference, (63.5ms) NMS
 The image with the result is saved in: runs\detect\exp8\input.png
Done. (0.481s)


YOLOR  v0.1-126-g84932d7 torch 2.1.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24575.5MB)

Model Summary: 792 layers, 151687420 parameters, 817020 gradients
C:\Users\covid\anaconda3\envs\localGPU\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [128]:
# ディレクトリのパス
directory_path = "C:/Users/covid/text_recognition/yolov7/runs/detect/"

# ディレクトリ内のサブディレクトリのリストを取得
subdirectories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

# サブディレクトリの中で一番新しいものを取得
newest_subdirectory = max(subdirectories, key=lambda d: os.path.getctime(os.path.join(directory_path, d)))

# 最新のサブディレクトリのパスを作成
newest_subdirectory_path = os.path.join(directory_path, newest_subdirectory)

# 新しいディレクトリに移動
os.chdir(newest_subdirectory_path)

# 画像ファイルの相対パスを指定
image_relative_path = "input.png"
# テキストファイルの相対パスを指定
text_file_relative_path = "labels/input.txt"

# 画像ファイルの絶対パスを作成
image_absolute_path = os.path.join(newest_subdirectory_path, image_relative_path)
# テキストファイルの絶対パスを作成
text_file_absolute_path = os.path.join(newest_subdirectory_path, text_file_relative_path)

# 画像をImageクラスのインスタンスに読み込む
image = Image.open(image_absolute_path)
# テキストファイルを読み込む
with open(text_file_absolute_path, 'r') as file:
    text_content = file.read()

In [129]:
# Process the text file
wid,hei = image.size
process_text_file(text_file_absolute_path)

In [132]:
#output corresponding to list format
if __name__ == '__main__':
    directory_path = 'C:/Users/covid/text_recognition/output'
    output_list = []

    file_list = [filename for filename in os.listdir(directory_path) if filename.endswith('.jpg')]
    file_list.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))

    for filename in file_list:
        input_file = os.path.join(directory_path, filename)
        output = read_ocr(service, input_file, 'ja')

        # 不要な文字（スペースとバックスラッシュ）を除去して一つの文字列に結合する
        cleaned_output = ''.join(line.strip().replace(' ', '').replace('/', '').replace('\n', '').replace('\\', '') for line in output)

        # 結果をリストに追加
        output_list.append(cleaned_output)

    # 結果のリストを出力
    for result in output_list:
        print(result)

はじめてのメカトロニクス実践
システムエンベデッド技
エンベデッド技術
boポイント代キーポイント代システム「開発のためのエンベデッド技
キーポイント代

ok0.35Excel




模型代数人鬥
-解析演習
--解析入門。
book0.47bookbook0.60bookbook0.35kboBOOKOokbook0.4キーポイントキーポイント最キーポイント代Excelsww解析入門!模型代教人門解析演習Structureandinterpretationbook0.43).400.28力学学bookObook0.4(book0.48スマートスピーカーズAプログラミングはじめてのパターパターン認識とパターン認識と実ロボットロボットロボット機械振動ためbock0.36込みシステムエンベデッド技はじめてのメカトロニクス実践エンベデッド技術エンベデッド技
0.471baStructureandInterpretation

力学
機械振動学
スマートスピーカー×Aプログラミング
パターン認識とはじめてのパター
パターンと
boc
Hermesか

